In [1]:
from Indexed_Dataset import *
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import random
from torch.utils.data import WeightedRandomSampler

In [2]:
torch.manual_seed(42)
np.random.seed(42)
# torch.use_deterministic_algorithms(True)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [3]:
train_arr = np.loadtxt("../data/no_coord_train.csv", dtype=np.float32, delimiter=",", skiprows=1)
# Split into training/validation sets
np.random.shuffle(train_arr)
valid_arr = train_arr[int(0.9 * train_arr.shape[0]):]
train_arr = train_arr[:int(0.9 * train_arr.shape[0])]
weights = train_arr[:, -1]

sampler = WeightedRandomSampler(weights=weights, num_samples=train_arr.shape[0], replacement=True)

train_set = Indexed_Dataset(arr=train_arr)
train_load = DataLoader(dataset=train_set,
                        batch_size=32,
#                         sampler=sampler,
                        shuffle=True,
                        # num_workers=1,
                        worker_init_fn=seed_worker,
                        generator=g,)

valid_set = Indexed_Dataset(arr=valid_arr)
valid_load = DataLoader(dataset=valid_set,
                        batch_size=32,
                        # num_workers=8,
                        shuffle=True,
                        worker_init_fn=seed_worker,
                        generator=g,)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
class TravelRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        origin_call_dim = 20
        origin_stand_dim = 5
        taxi_id_dim = 10
        self.embed_origin_call: nn.Module = nn.Embedding(29027, origin_call_dim, padding_idx=0)
        self.embed_origin_stand: nn.Module = nn.Embedding(64, origin_stand_dim)
        self.embed_taxi_id: nn.Module = nn.Embedding(448, taxi_id_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(9 + origin_call_dim + origin_stand_dim + taxi_id_dim, 1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000, 800),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(800, 1)
        )

    def forward(self, input: torch.Tensor):
        origin_call = self.embed_origin_call(input[:, 0].to(dtype=torch.int32))
        origin_stand = self.embed_origin_stand(input[:, 1].to(dtype=torch.int32))
        taxi_id = self.embed_taxi_id(input[:, 2].to(dtype=torch.int32))
        input = torch.cat((origin_call, origin_stand, taxi_id, input[:, 3:]), dim=1).to(dtype=torch.float32)
        input = self.feed_forward(input)
        return input

model = TravelRegressor().to(device)

In [7]:
model.eval()
model(train_set[0][0].unsqueeze(0).to(device)).size()
# train_set[0][0].unsqueeze(0).size()

torch.Size([1, 1])

In [32]:
lossfn = nn.MSELoss(reduction="mean")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [33]:
def weight_mse(pred, act):
    denom = torch.sum(act)
    return torch.sum(act * ((pred - act)**2 / len(act))) / denom

In [34]:
def validate():
    model.eval()
    loss = torch.tensor([0]).to(device, dtype=torch.float32)
    with torch.no_grad():
        for i, (entry, target) in enumerate(valid_load):
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry)
            loss += lossfn(preds, target)

    return loss /(i + 1)

In [37]:
rmse_loss = []
def train(num_iter: int):
    for epoch in range(num_iter):
        rolling_loss = 0
        entries = 0
        for i, (entry, target) in enumerate(train_load):
            model.train()
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry)

            optimizer.zero_grad()
            loss = lossfn(preds, target) # or weighted mse
            loss.backward()
            optimizer.step()
            rolling_loss += loss * entry.size(0)
            entries += entry.size(0)
            if (i + 1) % 500 == 0 or i == 0:
                rolling_loss /= entries
                print(f"[Epoch: {epoch + 1}]\t[Iter: {i + 1}]\t[RMSE: {torch.sqrt(rolling_loss)}]")
                print(f"Prediction std dev: {torch.std(preds).item()}")
                rmse_loss.append(torch.sqrt(rolling_loss).item())
                entries = 0
                rolling_loss = 0

        valid_loss = validate()
        print(f"[RMSE Validation: {torch.sqrt(valid_loss).item()}]")

In [38]:
train(10)

[Epoch: 1]	[Iter: 1]	[RMSE: 388.50921630859375]
Prediction std dev: 148.87001037597656
[Epoch: 1]	[Iter: 500]	[RMSE: 721.9417114257812]
Prediction std dev: 173.75711059570312
[Epoch: 1]	[Iter: 1000]	[RMSE: 752.738525390625]
Prediction std dev: 255.46144104003906
[Epoch: 1]	[Iter: 1500]	[RMSE: 791.73388671875]
Prediction std dev: 132.77206420898438


KeyboardInterrupt: 